## Description
This notebook demonstrates how to access global population grids and calculate the population for a given area for a given time.

If you don't have accurate population data, you can use these grids as an estimate.

The grids are found here: https://www.worldpop.org/geodata/listing?id=69

Estimated total number of people per grid-cell. The dataset is available to download in Geotiff format at a resolution of 3 arc (approximately 100m at the equator). The projection is Geographic Coordinate System, WGS84. The units are number of people per pixel with country totals adjusted to match the corresponding official United Nations population estimates that have been prepared by the Population Division of the Department of Economic and Social Affairs of the United Nations Secretariat (2019 Revision of World Population Prospects). The mapping approach is Random Forest-based dasymetric redistribution.

Region : Ghana

DOI : 10.5258/SOTON/WP00660

Date of production : 2020-02-01

WorldPop (www.worldpop.org - School of Geography and Environmental Science, University of Southampton; Department of Geography and Geosciences, University of Louisville; Departement de Geographie, Universite de Namur) and Center for International Earth Science Information Network (CIESIN), Columbia University (2018). Global High Resolution Population Denominators Project - Funded by The Bill and Melinda Gates Foundation (OPP1134076). https://dx.doi.org/10.5258/SOTON/WP00660



In [1]:
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg

In [2]:
def getFeatures(gdf):
    """Function to convert features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [3]:
#specify your area of interest

latitude = (6.801, 6.893)
longitude = (-1.440, -1.350)

miny, maxy = latitude
minx, maxx = longitude
bbox = box(minx, miny, maxx, maxy)
geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))
coords = getFeatures(geo)
print(coords)

[{'type': 'Polygon', 'coordinates': [[[-1.35, 6.801], [-1.35, 6.893], [-1.44, 6.893], [-1.44, 6.801], [-1.35, 6.801]]]}]


In [10]:
#specify the country code
cnt_code ='gha'

#specify the years
#time = ('2017-01-01', '2018-12-31')
start_year = 2000
end_year =2020
years = range(start_year,end_year+1)
pop_results={}

#open and subset a population grid for each year; then calc the population for a given year for a given aoi.
for year in years:
    country_pop_file_name =f'ftp://ftp.worldpop.org.uk/GIS/Population/Global_2000_2020/{year}/{cnt_code.upper()}/{cnt_code.lower()}_ppp_{year}_UNadj.tif'
    data = rasterio.open(country_pop_file_name)
    with rasterio.open(country_pop_file_name) as src:
        out_image, out_transform = rasterio.mask.mask(src, coords, nodata=0,crop=True)
        out_meta = src.meta
    pop_results[year]=int(out_image.sum())
print(pop_results.values())


dict_values([27364, 26699, 26898, 27724, 28606, 28614, 27621, 28308, 27760, 25020, 24344, 24356, 26211, 26789, 25335, 25845, 26007, 26569, 26204, 26225, 25861])


From GIS

2017:   26652

2018:   26292

From code:

2017:  26569

2018:  26204

In [13]:
#the population values for 2000-2020. Interesting how it starts higher.
print(pop_results.values())

dict_values([27364, 26699, 26898, 27724, 28606, 28614, 27621, 28308, 27760, 25020, 24344, 24356, 26211, 26789, 25335, 25845, 26007, 26569, 26204, 26225, 25861])


In [14]:
pop_results.keys()

dict_keys([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])